In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
import json
import datetime
import warnings
warnings.filterwarnings(action="ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import bs4
import requests
import time
import math
import random
from pyproj import Geod

In [2]:
# import cv2
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import QTimer, QTime, QThread, pyqtSignal
from PyQt5 import QtCore, QtWidgets, uic, QtGui
# import import_ipynb
# import cvlib as cv
import argparse
import numpy as np
import time
# from ledIndicator.LedIndicatorWidget import LedIndicator
import pandas as pd 
import json, os, requests

In [3]:
start_Form, _ = uic.loadUiType("dcsa_api_ui/start.ui")
get_Form, _ = uic.loadUiType("dcsa_api_ui/get_main.ui")
post_Form, _ = uic.loadUiType("dcsa_api_ui/post_main.ui")

In [4]:
# party_name = ""
# stakeholder = ""

In [5]:
# token_url = "https://u1hx677ico.auth.eu-west-1.amazoncognito.com/oauth2/token"

# #client (application) credentials on apim.byu.edu
# client_id = '6lmdehr3ti43059jnq79ofuslj'
# client_secret = '1ukglj96o0ch0dopgu90hu0lvi6009emhe3i67d8331qneuaco5k'

# #step A, B - single call with client credentials as the basic auth header - will return access_token
# data = {'grant_type': 'client_credentials'}

# access_token_response = requests.post(token_url, data=data, verify=False, allow_redirects=False, auth=(client_id, client_secret))
# tokens = json.loads(access_token_response.text)
# tokens["access_token"]

In [6]:
# token authorization
def token_authorization():
    token_url = "https://5tplhwckrm.auth.eu-west-1.amazoncognito.com/oauth2/token"

    #client (application) credentials on apim.byu.edu
    client_id = '2eu3rntgi9acc2s3l264j57ji7'
    client_secret = 'b5lgae9brcp2ne2lgglr5lsf65jbtcqnolu3ckijs0beffi1u93'

    #step A, B - single call with client credentials as the basic auth header - will return access_token
    data = {'grant_type': 'client_credentials'}

    access_token_response = requests.post(
        token_url,
        data=data,
        verify=False,
        allow_redirects=False,
        auth=(client_id, client_secret),
    )
    tokens = json.loads(access_token_response.text)
    return tokens["access_token"]

In [7]:
class StartWindow(QMainWindow, start_Form):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.show()
#         self.input_party_name.returnPressed.connect(self.send_partyname)
        self.pb_events.clicked.connect(self.getwindowStart) # TODO : class 밖에서 getwindow 시작하는 함수 만들어야 함
        self.pb_timestamps.clicked.connect(self.postwindowStart) # TODO : class 밖에서 postwindow 시작하는 함수 만들어야 함
#         global party_name 
#         global stakeholder
#         self.party_name = self.input_party_name.text()
#         self.stakeholder = self.combo_stakeholders.currentText()
        
#     def get_button_clicked(self):
# #        1. "현재 창 닫고"
# #        2. "get window 연다."
#     def post_button_clicked(self):
#         # 1. 현재 창 닫고
#         # 2. post window 연다.

#     def send_partyname(self):
#         global party_name        
#         party_name = self.input_party_name.text()
# #         self.close()
        
    def getwindowStart(self):
        self.close()
        self.getwindow = GetWindow()
#         self.getwindow.exec()
        
    def postwindowStart(self):
        self.close()
        global party_name
        global stakeholder
        party_name = self.input_party_name.text()
        stakeholder = self.combo_stakeholders.currentText()
        self.postwindow = PostWindow()
#         self.postwindow.exec()

In [8]:
class GetWindow(QMainWindow, get_Form) : 
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.show()
        self.cb_arri.setChecked(True)
        self.cb_cmlpl.setChecked(True)
        self.cb_depa.setChecked(True)
        self.cb_strt.setChecked(True)
        
        self.PB_GET.clicked.connect(self.get_btn_clicked)  # TODO : class 밖에서 param 불러오는 함수 만들어야 함
        
    def get_btn_clicked(self):
        param = self.send_get()
        api_call_response = self.api_call(param)
        print(api_call_response.json())
        with open('dcsa_test_Get.json', 'w', encoding="utf-8 sig") as file:
            json.dump(api_call_response.json(), file)
        self.close()
        
    def api_call(self, parameters):
        api_call_headers = api_header(mode="GET")
        added_param = parameter_for_url(parameters)
        url = events_url + added_param
        api_call_response = requests.get(events_url, headers=api_call_headers, verify=False)
        print(api_call_response)
#         with open('dcsa_test_Get.json', 'w', encoding="utf-8 sig") as file:
#             json.dump(api_call_response, file)
        return api_call_response
    
    # 꼭 다 선택해야 하는 건 아니다. 최소 1개만 있으면 된다. 
    def send_get(self):
        param = {}
        if not self.input_imo_no.text() == None or "":
            imo_no = self.input_imo_no.text()
            param["vesselIMONumber"] = imo_no
            
        if not self.input_ex_voy_no.text() == None or "":
            ex_voy_no = self.input_ex_voy_no.text()
            param["exportVoyageNumber"] = ex_voy_no
            
        if not self.input_carrier.text() == None or "":
            carrier = self.input_carrier.text()
            param["carrierServiceCode"] = carrier
            
        if not self.input_carrier.text() == None or "":
            UNLocationCode = self.un_code_result    
            param["UNLocationCode"] = UNLocationCode
            
        if not self.input_carrier.text() == None or "":
            operations_event_type_list = self.op_event_result
            param["operationsEventTypeCode"] = operations_event_type_list
 
        limit = int(self.limit_spinbox.value())
        param["limit"] = limit
        
        return param
    
    
    def un_code_result(self):
        loc_name = self.combo_un_code.currentText()
        unlocationcodes_df = pd.read_csv("unlocationcodes.csv")
        df_ = unlocationcodes_df[unlocationcodes_df["UN Location Name"] == loc_name]
        UNLocationCode = df_["UN Location Code"].values[0]        
        return UNLocationCode
        
    def op_event_result(self):
        checked = []
#         items = ""
        
        if self.cb_arri.isChecked():
            checked.append("ARRI")
        
        if self.cb_cmlpl.isChecked():
            checked.append("CMPL")
            
        if self.cb_depa.isChecked():
            checked.append("DEPA")
            
        if self.cb_strt.isChecked():
            checked.append("STRT")
            
        return checked
    
#     def combine(self):
#         param = send_get()
#         added_param = parameter_for_url(param)
#         UNLocationCode = un_code_result()
#         checked = op_event_result()
#         parameters = [added_param, UNLocationCode, checked]
        
#         return parameters

In [9]:
class PostWindow(QMainWindow, post_Form):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.show()
        self.pb_POST.clicked.connect(self.post_btn_clicked)  # TODO : class 밖에서 param 불러오는 함수 만들어야 함
#         self.party_name=party_name
#         self.stakeholder=stakeholder

    
    def vessel_imo_no_result(self):
#         assert self.input_vessel_code.text() == str
        vssl_code =self.input_vessel_code.text()
        vssl_code = str(vssl_code)
        return vssl_code
    
    def vessel_position_result(self):
#         assert self.input_vessel_position.text() == tuple
        position= self.input_vessel_position.text()
        position = position.split(",")
        latitude = position[0]
        longitude = position[1]
        return latitude, longitude
    
    def nan_into_null(self, VALUE):
        if VALUE != VALUE:
            VALUE = ""
        return VALUE
    
    def voyage_number(self):
        voyageNumber= self.input_vessel_voyage_number.text()
        return voyageNumber
    
    def berth_location(self):
        berth_no =self.combo_berth_no.currentText()
        T1_location = (128.82589744, 35.07801261)
        T2_location = (128.82503294, 35.07859456)
        T3_location = (128.82277395, 35.07831)
        OT_location = (None, None)
        if berth_no == "T1":
            berth_location_latitude, berth_location_longitude = T1_location
            return berth_no, berth_location_latitude, berth_location_longitude
        elif berth_no == "T2":
            berth_location_latitude, berth_location_longitude = T2_location
            return berth_no, berth_location_latitude, berth_location_longitude
        elif berth_no == "T3" :
            berth_location_latitude, berth_location_longitude = T3_location
            return berth_no, berth_location_latitude, berth_location_longitude
        else:
            berth_location_latitude, berth_location_longitude = OT_location
            return berth_no, berth_location_latitude, berth_location_longitude
    
    def post_btn_clicked(self):
        eventTimestamp = self.timestamp_result()
        publisherRole, operationsEventTypeCode, portCallPhaseTypeCode, facilityTypeCode, eventClassifierCode = self.choose_timestamp()
        delay_reason_code, delay_reason_name = self.delay_reason_result()
        eventTime = self.event_time_result()
        UNLocationCode = self.choose_un_loc_code()
        vessel_code = self.vessel_imo_no_result()
        vessel_latitude, vessel_longitude = self.vessel_position_result()
        voyageNumber = self.voyage_number()
        if publisherRole == "TR":
            berth_no, berth_location_latitude, berth_location_longitude = self.berth_location()
        else:
            berth_no, berth_location_latitude, berth_location_longitude = "","",""
#         if isVesselPositionNeeded == False or "false" or "False":
#             latitude, longitude = "", ""
#         portCallPhaseTypeCode = self.nan_into_null(portCallPhaseTypeCode)
        facilityTypeCode = self.nan_into_null(facilityTypeCode)
        if stakeholder == "Terminal":
            partyCode = "KRPUS"
            codeListName = 'TCL'
        else:
            partyCode = "MSK"
            codeListName = 'LCL'
            
            
        post_dict = {
            "publisher": {
                "partyName": party_name, 
                "taxReference1": "",
                "taxReference2": "",
                "publicKey": "",
                "address": {
                    "name": "",
                    "street": "",
                    "streetNumber": "",
                    "floor": "",
                    "postCode": "",
                    "city": "",
                    "stateRegion": "",
                    "country": ""
                },
                "identifyingCodes": [{
                    "codeListResponsibleAgencyCode": "306",
                    "DCSAResponsibleAgencyCode": "SMDG",
                    "partyCode": partyCode,
                    "codeListName": codeListName
                }]
            },
            "publisherRole": publisherRole,
            "vesselIMONumber": str(vessel_code),
            "UNLocationCode": "KRPUS",
            "facilityTypeCode": facilityTypeCode,
            "eventClassifierCode": eventClassifierCode,
            "operationsEventTypeCode": operationsEventTypeCode,
            "eventLocation": {
                "locationName": "PNIT No.1 Berth",
                "latitude": "132.1512",
                "longitude": "12.512",
                "UNLocationCode": "KRPUS",
                "facilityCode": "PNP",
                "facilityCodeListProvider": "SMDG",
                "address": {
                    "name": "",
                    "street": "",
                    "streetNumber": "",
                    "floor": "",
                    "postCode": "",
                    "city": "",
                    "stateRegion": "",
                    "country": ""
                }
            },
            "vesselPosition": {
                "latitude": vessel_latitude,
                "longitude": vessel_longitude
            },
            "modeOfTransport": "VESSEL",
            "portCallServiceTypeCode": None,
            "portCallPhaseTypeCode": portCallPhaseTypeCode,
            "eventDateTime": eventTime,
            "carrierVoyageNumber" : voyageNumber,
            "carrierServiceCode": "FE1",
            "transportCallSequenceNumber": 2,
            "remark": delay_reason_name,
            "delayReasonCode": delay_reason_code
        }  
        
        json_string = json.dumps(post_dict)
        print(json_string)
#         print("-")
#         json_string = json.loads(json_string)
#         print(json_string)
#         with open('dcsa_test.json', 'w', encoding="utf-8 sig") as file:
#             json.dump(json_string, file)
        
        api_call_headers = api_header(mode="POST", json=json_string)
        print("header", api_call_headers)
        api_call_response = requests.post(timestamps_url, data = json_string, headers=api_call_headers, verify=False)
        print('api_call_response', api_call_response)
        self.close()
        
        
    def timestamp_result(self):
        return self.combo_timestamp.currentText()
    
    def delay_reason_result(self):   
        delay_code_df = pd.read_csv("smdgdelayreasoncodes.csv")
        delay_reason_name = self.combo_delay_reason.currentText()
        if delay_reason_name == "N/A":
            delay_reason_code, delay_reason_name = None, None
            return delay_reason_code, delay_reason_name
        else:
            df_ = delay_code_df[delay_code_df["delay_reason_name"] == delay_reason_name]
            delay_reason_code = df_['delay_reason_code'].values[0]
            return str(delay_reason_code), str(delay_reason_name)
    
    def event_time_result(self):
        dt = self.datetime_event.dateTime()
        dt = dt.toString("yyyy-MM-dd HH:mm:ss")
        dt_split = dt.split(" ")
        eventTime = dt_split[0] + "T" + dt_split[1] + "+09:00"
        return eventTime
    
    def choose_timestamp(self):
        timestamp_df = pd.read_csv("timestampdefinitions.csv")
        TIMESTAMP = self.timestamp_result()
        df_ = timestamp_df[timestamp_df["timestampTypeName"] == TIMESTAMP]
        publisherRole = df_["publisherRole"].values[0]
        operationsEventTypeCode = df_["operationsEventTypeCode"].values[0]
        portCallPhaseTypeCode = df_["portCallPhaseTypeCode"].values[0]
#         portCallServiceTypeCode = df_["portCallServiceTypeCode"].values[0]
        facilityTypeCode = df_["facilityTypeCode"].values[0]
        eventClassifierCode = df_["eventClassifierCode"].values[0]
#         isVesselPositionNeeded = df_["isVesselPositionNeeded"].values[0]
#         portCallServiceTypeCode = "" if portCallServiceTypeCode == np.NaN or float(np.nan) else portCallServiceTypeCode
#         facilityTypeCode = "" if portCallServiceTypeCode == np.NaN or float(np.nan) else portCallServiceTypeCode
    
        
        return publisherRole, operationsEventTypeCode, portCallPhaseTypeCode, facilityTypeCode, eventClassifierCode
    
    def choose_un_loc_code(self):
        loc_name = self.combo_un_code.currentText()
        unlocationcodes_df = pd.read_csv("unlocationcodes.csv")
        df_ = unlocationcodes_df[unlocationcodes_df["UN Location Name"] == loc_name]
        UNLocationCode = df_["UN Location Code"].values[0]        
        return UNLocationCode
        

# 공통 변수들

In [10]:
# token_url = "https://u1hx677ico.auth.eu-west-1.amazoncognito.com/oauth2/token"

timestamps_url = "https://pnit.p6-busan-production.dcsa.org/jit/v1/timestamps"
events_url = "https://pnit.p6-busan-production.dcsa.org/jit/v1/events?"

# #client (application) credentials on apim.byu.edu
# client_id = '6lmdehr3ti43059jnq79ofuslj'
# client_secret = '1ukglj96o0ch0dopgu90hu0lvi6009emhe3i67d8331qneuaco5k'

# #step A, B - single call with client credentials as the basic auth header - will return access_token
# data = {'grant_type': 'client_credentials'}



## GET 전용

In [11]:
def parameter_for_url(param):
    if len(param) == 1:
        added_param = ''
        for key, value in param.items():
            added_param += str(key) + "="
            added_param += str(value)
    else:     
        added_param = ''
        i = 0
        for key, value in param.items():
            added_param += str(key) + "="
            added_param += str(value)
            i += 1
            if i < len(param):
                added_param += "&"
    return added_param

In [12]:
def api_header(mode, json=None):
    token = token_authorization()
    
    if mode == "GET":

        api_call_headers = {'Authorization': 'Bearer ' + token,    
                            'Host' : 'pnit.p6-busan-production.dcsa.org', 
                            'User-Agent' : 'PostmanRuntime/7.29.0', 
                            'Accept': '*/*', 
                            'Accept-Encoding' :  'gzip, deflate, br', 
                            'Connection' : 'keep-alive'}
        
    elif mode =="POST":
        api_call_headers = {
            'Authorization': 'Bearer ' + token, 
            "Content-Type" : 'application/json', 
            'Content-Length' : str(len(json)),
            'User-Agent' : 'PostmanRuntime/7.29.0',
            'Host' : 'pnit.p6-busan-production.dcsa.org', 
            'Accept': '*/*', 
            'Accept-Encoding' :  'gzip, deflate, br', 
            'Connection' : 'keep-alive'
        }        
        
    return api_call_headers



In [23]:
parameters = {'vesselIMONumber' : [9687540, 9687576]}
# parameters = {'vesselIMONumber' : 9687576}

In [24]:
api_call_headers = api_header(mode="GET")
added_param = parameter_for_url(parameters)
url = events_url + added_param
api_call_response = requests.get(events_url, headers=api_call_headers, verify=False)
# print(api_call_response)
#         with open('dcsa_test_Get.json', 'w', encoding="utf-8 sig") as file:
#             json.dump(api_call_response, file)
api_call_response.text

'[]'

In [22]:
# app = QApplication(sys.argv)

# startwindow = StartWindow()
# startwindow.show()
# app.exec_()